## Original Implemntation

In [ ]:
import argparse
import datetime
import fnmatch
import os
import pickle
import re
from time import time
from typing import List

import tensorflow.keras
import numpy
import numpy as np
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from ClassWeightCalculator import ClassWeightCalculator
from reporting import TelegramNotifier, GoogleSpreadsheetReporter, sklearn_reporting
from reporting.TrainingHistoryPlotter import TrainingHistoryPlotter
from datasets.TrainingDatasetProvider import TrainingDatasetProvider
from datasets.DirectoryIteratorWithBoundingBoxes import DirectoryIteratorWithBoundingBoxes
from models.ConfigurationFactory import ConfigurationFactory


def train_model(dataset_directory: str, model_name: str, stroke_thicknesses: List[int],
                width: int, height: int,
                staff_line_vertical_offsets: List[int], training_minibatch_size: int,
                optimizer: str, dynamic_learning_rate_reduction: bool, use_fixed_canvas: bool, datasets: List[str],
                class_weights_balancing_method: str,
                save_after_every_epoch: bool,
                resume_from_checkpoint: str,
                send_telegram_messages: bool):
    image_dataset_directory = os.path.join(dataset_directory, "images")

    bounding_boxes = None
    bounding_boxes_cache = os.path.join(dataset_directory, "bounding_boxes.txt")

    print("Loading configuration and data-readers...")
    start_time = time()

    number_of_classes = len(os.listdir(os.path.join(image_dataset_directory, "training")))
    training_configuration = ConfigurationFactory.get_configuration_by_name(model_name, optimizer, width, height,
                                                                            training_minibatch_size, number_of_classes)
    if training_configuration.performs_localization() and bounding_boxes is None:
        # Try to unpickle
        with open(bounding_boxes_cache, "rb") as cache:
            bounding_boxes = pickle.load(cache)

    if not training_configuration.performs_localization():
        bounding_boxes = None

    train_generator = ImageDataGenerator(rotation_range=training_configuration.rotation_range,
                                         zoom_range=training_configuration.zoom_range
                                         )
    training_data_generator = DirectoryIteratorWithBoundingBoxes(
        directory=os.path.join(image_dataset_directory, "training"),
        image_data_generator=train_generator,
        target_size=(training_configuration.input_image_rows,
                     training_configuration.input_image_columns),
        batch_size=training_configuration.training_minibatch_size,
        bounding_boxes=bounding_boxes,
    )
    training_steps_per_epoch = np.math.ceil(training_data_generator.samples / training_data_generator.batch_size)

    validation_generator = ImageDataGenerator()
    validation_data_generator = DirectoryIteratorWithBoundingBoxes(
        directory=os.path.join(image_dataset_directory, "validation"),
        image_data_generator=validation_generator,
        target_size=(
            training_configuration.input_image_rows,
            training_configuration.input_image_columns),
        batch_size=training_configuration.training_minibatch_size,
        bounding_boxes=bounding_boxes)
    validation_steps_per_epoch = np.math.ceil(validation_data_generator.samples / validation_data_generator.batch_size)

    test_generator = ImageDataGenerator()
    test_data_generator = DirectoryIteratorWithBoundingBoxes(
        directory=os.path.join(image_dataset_directory, "test"),
        image_data_generator=test_generator,
        target_size=(training_configuration.input_image_rows,
                     training_configuration.input_image_columns),
        batch_size=training_configuration.training_minibatch_size,
        shuffle=False,
        bounding_boxes=bounding_boxes)
    test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

    model:Model = training_configuration.classifier()
    model.summary()

    print("Model {0} created.".format(training_configuration.name()))

    initial_epoch = 0
    if resume_from_checkpoint:
        # Try to parse epoch from checkpoint filename. Checkpoint files written by this program
        # are of the form <start>_<configname>-<epoch>.h5.
        # The regular expression assumes there are no dashes in configname, otherwise
        # the initial epoch will just be 0. That is harmless unless you have parameters that
        # depend on the epoch.
        m = re.match('[\d-]+_[^-]+-(\d+).h5', resume_from_checkpoint)
        if m and m.groups() and len(m.groups() == 1):
            initial_epoch = int(m.groups()[0]) + 1
        # This will not restore parameters that are adapted dynamically during training since
        # afaik not all of them get saved to the model checkpoint.
        model.load_weights(resume_from_checkpoint)
        print("Model {0} weights loaded from checkpoint {1}. Training will resume from epoch {2}".format(
            training_configuration.name(),
            resume_from_checkpoint,
            initial_epoch))

    print(training_configuration.summary())

    start_of_training = datetime.date.today()

    monitor_variable = 'val_accuracy'
    if training_configuration.performs_localization():
        monitor_variable = 'val_output_class_accuracy'

    best_model_path = "{0}_{1}".format(start_of_training, training_configuration.name())
    if save_after_every_epoch:
        model_checkpoint = ModelCheckpoint(best_model_path + "-{epoch:02d}.h5", monitor=monitor_variable,
                save_best_only=True, verbose=1, save_freq='epoch')
    else:
        model_checkpoint = ModelCheckpoint(best_model_path+".h5", monitor=monitor_variable,
                save_best_only=True, verbose=1)
    early_stop = EarlyStopping(monitor=monitor_variable,
                               patience=training_configuration.number_of_epochs_before_early_stopping,
                               verbose=1)
    learning_rate_reduction = ReduceLROnPlateau(monitor=monitor_variable,
                                                patience=training_configuration.number_of_epochs_before_reducing_learning_rate,
                                                verbose=1,
                                                factor=training_configuration.learning_rate_reduction_factor,
                                                min_lr=training_configuration.minimum_learning_rate)
    tensorboard_callback = TensorBoard(
            log_dir="./logs/{0}_{1}/".format(start_of_training, training_configuration.name()))
    if dynamic_learning_rate_reduction:
        callbacks = [model_checkpoint, early_stop, tensorboard_callback, learning_rate_reduction]
    else:
        print("Learning-rate reduction on Plateau disabled")
        callbacks = [model_checkpoint, early_stop, tensorboard_callback]

    class_weight_calculator = ClassWeightCalculator()
    class_weights = class_weight_calculator.calculate_class_weights(image_dataset_directory,
                                                                    method=class_weights_balancing_method,
                                                                    class_indices=training_data_generator.class_indices)
    if class_weights_balancing_method is not None:
        print("Using {0} method for obtaining class weights to compensate for an unbalanced dataset.".format(
            class_weights_balancing_method))

    print("Training on dataset...")
    history = model.fit(
        x=training_data_generator,
        steps_per_epoch=training_steps_per_epoch,
        epochs=training_configuration.number_of_epochs,
        callbacks=callbacks,
        validation_data=validation_data_generator,
        validation_steps=validation_steps_per_epoch,
        class_weight=class_weights,
        initial_epoch=initial_epoch
    )

    best_model = None
    if not save_after_every_epoch:
        print("Loading best model from check-point and testing...")
        best_model = tensorflow.keras.models.load_model(best_model_path + '.h5')
    else:
        print("Loading latest model from check-point and testing...")
        latest_mtime = 0
        latest_file = None
        pattern = best_model_path + '*.h5'
        for f in os.listdir('.'):
            if fnmatch.fnmatch(f, pattern):
                fd = os.open(f, os.O_RDONLY)
                st = os.fstat(fd)
                mt = st.st_mtime
                if not latest_file or mt > latest_mtime:
                    latest_mtime = mt
                    latest_file = f

        print('latest model file is {0}'.format(latest_file))
        best_model = tensorflow.keras.models.load_model(latest_file)

    test_data_generator.reset()
    file_names = test_data_generator.filenames
    class_labels = os.listdir(os.path.join(image_dataset_directory, "test"))
    # Notice that some classes have so few elements, that they are not present in the test-set and do not
    # appear in the final report. To obtain the correct classes, we have to enumerate all non-empty class
    # directories inside the test-folder and use them as labels
    names_of_classes_with_test_data = [
        class_name for class_name in class_labels
        if os.listdir(os.path.join(image_dataset_directory, "test", class_name))]
    true_classes = test_data_generator.classes
    predictions = best_model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
    if training_configuration.performs_localization():
        predicted_classes = numpy.argmax(predictions[0], axis=1)
    else:
        predicted_classes = numpy.argmax(predictions, axis=1)

    test_data_generator.reset()
    evaluation = best_model.evaluate_generator(test_data_generator, steps=test_steps_per_epoch)
    classification_accuracy = 0

    print("Reporting classification statistics with micro average")
    report = sklearn_reporting.classification_report(true_classes, predicted_classes, digits=3,
                                                     target_names=names_of_classes_with_test_data, average='micro')
    print(report)

    print("Reporting classification statistics with macro average")
    report = sklearn_reporting.classification_report(true_classes, predicted_classes, digits=3,
                                                     target_names=names_of_classes_with_test_data, average='macro')
    print(report)

    print("Reporting classification statistics with weighted average")
    report = sklearn_reporting.classification_report(true_classes, predicted_classes, digits=3,
                                                     target_names=names_of_classes_with_test_data, average='weighted'
                                                     )
    print(report)

    indices_of_misclassified_files = [i for i, e in enumerate(true_classes - predicted_classes) if e != 0]
    misclassified_files = [file_names[i] for i in indices_of_misclassified_files]
    misclassified_files_actual_prediction_indices = [predicted_classes[i] for i in indices_of_misclassified_files]
    misclassified_files_actual_prediction_classes = [class_labels[i] for i in
                                                     misclassified_files_actual_prediction_indices]
    print("Misclassified files:")
    for i in range(len(misclassified_files)):
        print("\t{0} is incorrectly classified as {1}".format(misclassified_files[i],
                                                              misclassified_files_actual_prediction_classes[i]))

    for i in range(len(best_model.metrics_names)):
        current_metric = best_model.metrics_names[i]
        print("{0}: {1:.5f}".format(current_metric, evaluation[i]))
        if current_metric == 'accuracy' or current_metric == 'output_class_acc':
            classification_accuracy = evaluation[i]
    print("Total Accuracy: {0:0.5f}%".format(classification_accuracy * 100))
    print("Total Error: {0:0.5f}%".format((1 - classification_accuracy) * 100))

    end_time = time()
    execution_time_in_seconds = round(end_time - start_time)
    print("Execution time: {0:.1f}s".format(end_time - start_time))

    training_result_image = "{1}_{0}_{2:.1f}p.png".format(training_configuration.name(), start_of_training,
                                                          classification_accuracy * 100)
    TrainingHistoryPlotter.plot_history(history, training_result_image)

    datasets_string = str.join(",", datasets)
    notification_message = "Training on {0} dataset with model {1} finished. " \
                           "Accuracy: {2:0.5f}%".format(datasets_string, model_name, classification_accuracy * 100)
    if send_telegram_messages:
        TelegramNotifier.send_message_via_telegram(notification_message, training_result_image)
    else:
        print(notification_message)

    dataset_size = training_data_generator.samples + validation_data_generator.samples + test_data_generator.samples
    stroke_thicknesses_string = ",".join(map(str, stroke_thicknesses))
    staff_line_vertical_offsets_string = ",".join(map(str, staff_line_vertical_offsets))
    image_sizes = "{0}x{1}px".format(training_configuration.input_image_rows,
                                     training_configuration.input_image_columns)
    data_augmentation = "{0}% zoom, {1}° rotation".format(int(training_configuration.zoom_range * 100),
                                                          training_configuration.rotation_range)
    today = "{0:02d}.{1:02d}.{2}".format(start_of_training.day, start_of_training.month, start_of_training.year)
    balancing_method = "None" if class_weights_balancing_method is None else class_weights_balancing_method

    GoogleSpreadsheetReporter.append_result_to_spreadsheet(dataset_size=dataset_size, image_sizes=image_sizes,
                                                           stroke_thicknesses=stroke_thicknesses_string,
                                                           staff_lines=staff_line_vertical_offsets_string,
                                                           model_name=model_name, data_augmentation=data_augmentation,
                                                           optimizer=optimizer,
                                                           early_stopping=training_configuration.number_of_epochs_before_early_stopping,
                                                           reduction_patience=training_configuration.number_of_epochs_before_reducing_learning_rate,
                                                           learning_rate_reduction_factor=training_configuration.learning_rate_reduction_factor,
                                                           minibatch_size=training_minibatch_size,
                                                           initialization=training_configuration.initialization,
                                                           initial_learning_rate=training_configuration.get_initial_learning_rate(),
                                                           accuracy=classification_accuracy,
                                                           date=today,
                                                           use_fixed_canvas=use_fixed_canvas,
                                                           datasets=datasets_string,
                                                           execution_time_in_seconds=execution_time_in_seconds,
                                                           balancing_method=balancing_method)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.register("type", "bool", lambda v: v.lower() == "true")
    parser.add_argument("--dataset_directory", type=str, default="data",
                        help="The directory, that is used for storing the images during training")
    parser.add_argument("--model_name", type=str, default="res_net_4",
                        help="The model used for training the network. Run ListAvailableConfigurations.ps1 or "
                             "models/ConfigurationFactory.py to get a list of all available configurations")

    parser.add_argument("--use_existing_dataset_directory", dest="delete_and_recreate_dataset_directory",
                        action='store_false',
                        help="Whether to delete and recreate the dataset-directory (by downloading the appropriate "
                             "files from the internet, extracting and generating images) or simply use whatever data "
                             "currently is inside of that directory.")
    parser.set_defaults(delete_and_recreate_dataset_directory=True)

    parser.add_argument("--minibatch_size", default=16, type=int,
                        help="Size of the minibatches for training, typically one of 8, 16, 32, 64 or 128")
    parser.add_argument("--optimizer", default="Adadelta", type=str,
                        help="The optimizer used for the training, can be SGD, Adam or Adadelta")

    parser.add_argument("--no_dynamic_learning_rate_reduction", dest="dynamic_learning_rate_reduction",
                        action="store_false",
                        help="True, if the learning rate should not be scheduled to be reduced on a plateau.")
    parser.set_defaults(dynamic_learning_rate_reduction=True)
    parser.add_argument("--class_weights_balancing_method", default=None, type=str,
                        help="The optional weight balancing method for handling unbalanced datasets. If provided,"
                             "valid choices are simple or skBalance. 'simple' uses 1/sqrt(#samples_per_class) as "
                             "weights for samples from each class to compensate for classes that are underrepresented."
                             "'skBalance' uses the Python SkLearn module to calculate more sophisticated weights.")
    parser.add_argument("--telegram_messages", dest="send_telegram_messages", action="store_true",
                        help="Send messages via telegram")
    parser.set_defaults(send_telegram_messages=False)
    parser.add_argument("--save_after_every_epoch", dest="save_after_every_epoch", action="store_true",
                        help="Write a checkpoint after every epoch")
    parser.set_defaults(save_after_every_epoch=False)
    parser.add_argument("--resume_from_checkpoint", dest="resume_from_checkpoint", default=None, type=str,
                        help="Load checkpoint from file specified.")

    TrainingDatasetProvider.add_arguments_for_training_dataset_provider(parser)

    flags, unparsed = parser.parse_known_args()

    offsets = []
    if flags.offsets != "":
        offsets = [int(o) for o in flags.offsets.split(',')]
    stroke_thicknesses_for_generated_symbols = [int(s) for s in flags.stroke_thicknesses.split(',')]

    if flags.datasets == "":
        raise Exception("No dataset selected. Specify the dataset for the training via the --dataset parameter")
    datasets = flags.datasets.split(',')

    if flags.delete_and_recreate_dataset_directory:
        training_dataset_provider = TrainingDatasetProvider(flags.dataset_directory)
        training_dataset_provider.recreate_and_prepare_datasets_for_training(
            datasets=datasets, width=flags.width,
            height=flags.height,
            use_fixed_canvas=flags.use_fixed_canvas,
            stroke_thicknesses_for_generated_symbols=stroke_thicknesses_for_generated_symbols,
            staff_line_spacing=flags.staff_line_spacing,
            staff_line_vertical_offsets=offsets,
            random_position_on_canvas=flags.random_position_on_canvas)
        training_dataset_provider.resize_all_images_to_fixed_size(flags.width, flags.height)
        training_dataset_provider.split_dataset_into_training_validation_and_test_set()

    train_model(dataset_directory=flags.dataset_directory,
                model_name=flags.model_name,
                stroke_thicknesses=stroke_thicknesses_for_generated_symbols,
                width=flags.width,
                height=flags.height,
                staff_line_vertical_offsets=offsets,
                training_minibatch_size=flags.minibatch_size,
                optimizer=flags.optimizer,
                dynamic_learning_rate_reduction=flags.dynamic_learning_rate_reduction,
                use_fixed_canvas=flags.use_fixed_canvas,
                datasets=datasets,
                class_weights_balancing_method=flags.class_weights_balancing_method,
                save_after_every_epoch=flags.save_after_every_epoch,
                resume_from_checkpoint=flags.resume_from_checkpoint,
                send_telegram_messages=flags.send_telegram_messages)

    # To run in in python console
    # dataset_directory = 'data'
    # model_name = 'res_net_3_small'
    # delete_and_recreate_dataset_directory = True
    # stroke_thicknesses = [3]
    # width = 96
    # height = 192
    # staff_line_vertical_offsets = None
    # staff_line_spacing = 14
    # training_minibatch_size = 32
    # optimizer = 'Adadelta'
    # dynamic_learning_rate_reduction = True

## Pytorch Implmentation

In [ ]:
import os
import time
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from datetime import datetime
import re
from typing import List

# Import your custom modules
from ClassWeightCalculator import ClassWeightCalculator
from reporting import TelegramNotifier, sklearn_reporting
from reporting.TrainingHistoryPlotter import TrainingHistoryPlotter
from datasets.TrainingDatasetProvider import TrainingDatasetProvider
from models.ConfigurationFactory import ConfigurationFactory


def train_model(dataset_directory: str, model_name: str, stroke_thicknesses: List[int],
                width: int, height: int,
                staff_line_vertical_offsets: List[int], training_minibatch_size: int,
                optimizer_name: str, dynamic_learning_rate_reduction: bool, use_fixed_canvas: bool, datasets: List[str],
                class_weights_balancing_method: str,
                save_after_every_epoch: bool,
                resume_from_checkpoint: str):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    start_of_training = datetime.now()
    start_time = time.time()

    # Paths and Configurations
    image_dataset_directory = os.path.join(dataset_directory, "images")
    bounding_boxes_cache = os.path.join(dataset_directory, "bounding_boxes.txt")
    bounding_boxes = None

    print("Loading configuration and data-readers...")
    number_of_classes = len(os.listdir(os.path.join(image_dataset_directory, "training")))
    training_configuration = ConfigurationFactory.get_configuration_by_name(
        model_name, optimizer_name, width, height, training_minibatch_size, number_of_classes
    )

    if training_configuration.performs_localization():
        # Load Bounding Boxes
        if os.path.exists(bounding_boxes_cache):
            with open(bounding_boxes_cache, "rb") as cache:
                bounding_boxes = pickle.load(cache)
        else:
            print("Bounding boxes cache not found.")
    else:
        bounding_boxes = None

    # Data Augmentation and Preprocessing
    train_transform = transforms.Compose([
        transforms.Resize((height, width)),
        transforms.RandomRotation(training_configuration.rotation_range),
        transforms.RandomResizedCrop((height, width), scale=(1.0 - training_configuration.zoom_range, 1.0)),
        transforms.ToTensor(),
    ])
    val_test_transform = transforms.Compose([
        transforms.Resize((height, width)),
        transforms.ToTensor(),
    ])

    # Datasets and Loaders
    if bounding_boxes is not None:
        train_dataset = DirectoryIteratorWithBoundingBoxes(directory=os.path.join(image_dataset_directory, "training"),,
                                                            bounding_boxes=bounding_boxes,
                                                            target_size=(192, 96),
                                                            transform=train_transform
                                                        )
        
        val_dataset = DirectoryIteratorWithBoundingBoxes(directory=os.path.join(image_dataset_directory, "validation"),
                                                            bounding_boxes=bounding_boxes,
                                                            target_size=(192, 96),
                                                            transform=val_test_transform
                                                        )
        test_dataset = DirectoryIteratorWithBoundingBoxes(directory=os.path.join(image_dataset_directory, "test"),
                                                            bounding_boxes=bounding_boxes,
                                                            target_size=(192, 96),
                                                            transform=val_test_transform
                                                        )
    else:
        train_dataset = ImageFolder(os.path.join(image_dataset_directory, "training"), transform=train_transform)
        val_dataset = ImageFolder(os.path.join(image_dataset_directory, "validation"), transform=val_test_transform)
        test_dataset = ImageFolder(os.path.join(image_dataset_directory, "test"), transform=val_test_transform)

    train_loader = DataLoader(train_dataset, batch_size=training_minibatch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=training_minibatch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=training_minibatch_size, shuffle=False)

    # Model, Optimizer, and Loss
    model = training_configuration.classifier().to(device)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=training_configuration.get_initial_learning_rate())

    # Learning Rate Scheduler
    if dynamic_learning_rate_reduction:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                         patience=training_configuration.number_of_epochs_before_reducing_learning_rate,
                                                         factor=training_configuration.learning_rate_reduction_factor,
                                                         min_lr=training_configuration.minimum_learning_rate)
    else:
        scheduler = None

    # Class Weights and Loss
    class_weight_calculator = ClassWeightCalculator()
    class_weights = class_weight_calculator.calculate_class_weights(
        image_dataset_directory,
        method=class_weights_balancing_method,
        class_indices=train_dataset.class_to_idx
    )
    if class_weights_balancing_method is not None:
        print(f"Using {class_weights_balancing_method} method for obtaining class weights to compensate for an unbalanced dataset.")
    class_weights_tensor = torch.tensor(list(class_weights.values())).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    # TensorBoard
    writer = SummaryWriter(log_dir=f"./logs/{start_of_training}_{model_name}")

    # Resume from Checkpoint
    initial_epoch = 0
    if resume_from_checkpoint:
        # Parse initial epoch from checkpoint filename
        m = re.match(r'.*_(\d+).pth', resume_from_checkpoint)
        if m:
            initial_epoch = int(m.group(1))
        checkpoint = torch.load(resume_from_checkpoint)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(f"Resumed training from checkpoint: {resume_from_checkpoint}, starting from epoch {initial_epoch}")

    print(training_configuration.summary())

    # Early Stopping
    early_stopping_counter = 0
    best_val_accuracy = 0.0

    # Training Loop
    for epoch in range(initial_epoch, training_configuration.number_of_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch in train_loader:
            if bounding_boxes is not None:
                images, labels, _ = batch 
            else:
                images, labels = batch
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_accuracy = 100.0 * correct / total
        writer.add_scalar("Loss/Train", running_loss / len(train_loader), epoch)
        writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
        print(f"Epoch {epoch + 1}/{training_configuration.number_of_epochs}, Train Loss: {running_loss / len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

        # Validation Phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in val_loader:
                if bounding_boxes is not None:
                    images, labels, _ = batch
                else:
                    images, labels = batch
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_accuracy = 100.0 * correct / total
        writer.add_scalar("Loss/Validation", val_loss / len(val_loader), epoch)
        writer.add_scalar("Accuracy/Validation", val_accuracy, epoch)
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        if scheduler:
            scheduler.step(val_accuracy)

        # Early Stopping Logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            early_stopping_counter = 0
            # Save best model
            best_model_path = f"{start_of_training.date()}_{model_name}_best.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, best_model_path)
            print(f"Saved best model to {best_model_path}")
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= training_configuration.number_of_epochs_before_early_stopping:
                print("Early stopping triggered")
                break

        # Save after every epoch if required
        if save_after_every_epoch:
            checkpoint_path = f"{start_of_training.date()}_{model_name}_epoch_{epoch + 1}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)
            print(f"Saved checkpoint to {checkpoint_path}")

    # Load the best model for testing
    print("Loading best model for testing...")
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Test Phase
    model.eval()
    correct = 0
    total = 0
    predictions, true_classes = [], []
    file_names = []
    misclassified_files = []
    class_labels = test_dataset.classes

    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            if bounding_boxes is not None:
                images, labels, _ = batch
            else:
                images, labels = batch
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            predictions.extend(predicted.cpu().numpy())
            true_classes.extend(labels.cpu().numpy())
            # Get file names
            batch_samples = test_dataset.samples[batch_idx * training_minibatch_size: (batch_idx + 1) * training_minibatch_size]
            batch_file_names = [s[0] for s in batch_samples]
            file_names.extend(batch_file_names)

            # Record misclassified files
            for idx in range(len(labels)):
                if labels[idx] != predicted[idx]:
                    misclassified_files.append((batch_file_names[idx], class_labels[predicted[idx]]))

    classification_accuracy = 100.0 * correct / total
    print(f"Test Accuracy: {classification_accuracy:.2f}%")

    print("Reporting classification statistics with micro average")
    report = sklearn_reporting.classification_report(true_classes, predictions, digits=3,
                                                     target_names=class_labels, average='micro')
    print(report)

    print("Reporting classification statistics with macro average")
    report = sklearn_reporting.classification_report(true_classes, predictions, digits=3,
                                                     target_names=class_labels, average='macro')
    print(report)

    print("Reporting classification statistics with weighted average")
    report = sklearn_reporting.classification_report(true_classes, predictions, digits=3,
                                                     target_names=class_labels, average='weighted')
    print(report)

    print("Misclassified files:")
    for file_path, predicted_class in misclassified_files:
        print(f"\t{os.path.basename(file_path)} is incorrectly classified as {predicted_class}")

    end_time = time.time()
    execution_time_in_seconds = round(end_time - start_time)
    print(f"Execution time: {execution_time_in_seconds:.1f}s")

    # Plot Training History
    # Assuming TrainingHistoryPlotter can work with the logged data
    # You might need to adjust this function to read data from TensorBoard logs
    training_result_image = f"{start_of_training.date()}_{training_configuration.name()}_{classification_accuracy:.1f}p.png"
    TrainingHistoryPlotter.plot_history(writer, training_result_image)

    datasets_string = ','.join(datasets)
    notification_message = f"Training on {datasets_string} dataset with model {model_name} finished. Accuracy: {classification_accuracy:.5f}%"
    print(notification_message)

    writer.close()


In [ ]:

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.register("type", "bool", lambda v: v.lower() == "true")
    parser.add_argument("--dataset_directory", type=str, default="data",
                        help="The directory, that is used for storing the images during training")
    parser.add_argument("--model_name", type=str, default="res_net_4",
                        help="The model used for training the network. Run ListAvailableConfigurations.ps1 or "
                             "models/ConfigurationFactory.py to get a list of all available configurations")

    parser.add_argument("--use_existing_dataset_directory", dest="delete_and_recreate_dataset_directory",
                        action='store_false',
                        help="Whether to delete and recreate the dataset-directory (by downloading the appropriate "
                             "files from the internet, extracting and generating images) or simply use whatever data "
                             "currently is inside of that directory.")
    parser.set_defaults(delete_and_recreate_dataset_directory=True)

    parser.add_argument("--minibatch_size", default=16, type=int,
                        help="Size of the minibatches for training, typically one of 8, 16, 32, 64 or 128")
    parser.add_argument("--optimizer", default="Adadelta", type=str,
                        help="The optimizer used for the training, can be SGD, Adam or Adadelta")

    parser.add_argument("--no_dynamic_learning_rate_reduction", dest="dynamic_learning_rate_reduction",
                        action="store_false",
                        help="True, if the learning rate should not be scheduled to be reduced on a plateau.")
    parser.set_defaults(dynamic_learning_rate_reduction=True)
    parser.add_argument("--class_weights_balancing_method", default=None, type=str,
                        help="The optional weight balancing method for handling unbalanced datasets. If provided,"
                             "valid choices are simple or skBalance. 'simple' uses 1/sqrt(#samples_per_class) as "
                             "weights for samples from each class to compensate for classes that are underrepresented."
                             "'skBalance' uses the Python SkLearn module to calculate more sophisticated weights.")
    parser.add_argument("--save_after_every_epoch", dest="save_after_every_epoch", action="store_true",
                        help="Write a checkpoint after every epoch")
    parser.set_defaults(save_after_every_epoch=False)
    parser.add_argument("--resume_from_checkpoint", dest="resume_from_checkpoint", default=None, type=str,
                        help="Load checkpoint from file specified.")

    TrainingDatasetProvider.add_arguments_for_training_dataset_provider(parser)

    flags, unparsed = parser.parse_known_args()

    offsets = []
    if flags.offsets != "":
        offsets = [int(o) for o in flags.offsets.split(',')]
    stroke_thicknesses_for_generated_symbols = [int(s) for s in flags.stroke_thicknesses.split(',')]

    if flags.datasets == "":
        raise Exception("No dataset selected. Specify the dataset for the training via the --dataset parameter")
    datasets = flags.datasets.split(',')

    if flags.delete_and_recreate_dataset_directory:
        training_dataset_provider = TrainingDatasetProvider(flags.dataset_directory)
        training_dataset_provider.recreate_and_prepare_datasets_for_training(
            datasets=datasets, width=flags.width,
            height=flags.height,
            use_fixed_canvas=flags.use_fixed_canvas,
            stroke_thicknesses_for_generated_symbols=stroke_thicknesses_for_generated_symbols,
            staff_line_spacing=flags.staff_line_spacing,
            staff_line_vertical_offsets=offsets,
            random_position_on_canvas=flags.random_position_on_canvas)
        training_dataset_provider.resize_all_images_to_fixed_size(flags.width, flags.height)
        training_dataset_provider.split_dataset_into_training_validation_and_test_set()

    train_model(dataset_directory=flags.dataset_directory,
                model_name=flags.model_name,
                stroke_thicknesses=stroke_thicknesses_for_generated_symbols,
                width=flags.width,
                height=flags.height,
                staff_line_vertical_offsets=offsets,
                training_minibatch_size=flags.minibatch_size,
                optimizer=flags.optimizer,
                dynamic_learning_rate_reduction=flags.dynamic_learning_rate_reduction,
                use_fixed_canvas=flags.use_fixed_canvas,
                datasets=datasets,
                class_weights_balancing_method=flags.class_weights_balancing_method,
                save_after_every_epoch=flags.save_after_every_epoch,
                resume_from_checkpoint=flags.resume_from_checkpoint,
                send_telegram_messages=flags.send_telegram_messages)
